In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
from matplotlib import rc
from matplotlib.ticker import MaxNLocator
from matplotlib.pyplot import figure
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [ ]:
def generateLANL(mass, charge, a, N_gamma=2.5e5): # num of layers
    N_pot = 5.9e22 
    alpha = 1.0 / 137
    
    m_e = 0.00051
    m_pi = 0.135
    m_eta = 0.548 * 0.9

    c_pi = 0.115
    c_eta = c_pi/30

    branch_pi = 0.98
    branch_eta = 0.39

    ageo10m = np.full(np.shape(mass), 8e-5)
    ageo35m = np.full(np.shape(mass), 8e-6)

    ageo10m_eta = np.full(np.shape(mass), 0.75e-3)
    ageo35m_eta = np.full(np.shape(mass), 0.6e-4)

    def I_3(z, x):
        return 2 / (3 * 3.14) * ((1 - 4 * x / z) ** 0.5) * ((1 - z) ** 3) * (2 * x + z) / z ** 2

    def dl(x):
        return quad(I_3, 4 * x, 1, args=(x))[0]  # pass x to I
    v_dl = np.vectorize(dl)

    pi = np.zeros(np.shape(mass))
    eta = np.zeros(np.shape(mass))

    pi[mass < m_pi / 2] = (1 - np.exp(-N_gamma * charge[mass < m_pi / 2] ** 2)) ** a * 2 * c_pi * branch_pi * alpha * N_pot * ageo10m[mass < m_pi/2] * v_dl(mass[mass < m_pi / 2] ** 2 / m_pi ** 2) * charge[mass < m_pi / 2] ** 2

    eta[mass < m_eta / 2] = (1 - np.exp(-N_gamma * charge[mass < m_eta / 2] ** 2)) ** a * 2 * c_eta * branch_eta * alpha * N_pot * ageo10m_eta[mass < m_eta/2] * v_dl(mass[mass < m_eta / 2] ** 2 / m_eta ** 2) * charge[mass < m_eta / 2] ** 2
    
    sensitivity10m = pi + eta

    pi = np.zeros(np.shape(mass))
    eta = np.zeros(np.shape(mass))

    pi[mass < m_pi / 2] = (1 - np.exp(-N_gamma * charge[mass < m_pi / 2] ** 2)) ** a * 2 * c_pi * branch_pi * alpha * N_pot * ageo35m[mass < m_pi/2] * v_dl(mass[mass < m_pi / 2] ** 2 / m_pi ** 2) * charge[mass < m_pi / 2] ** 2

    eta[mass < m_eta / 2] = (1 - np.exp(-N_gamma * charge[mass < m_eta / 2] ** 2)) ** a * 2 * c_eta * branch_eta * alpha * N_pot * ageo35m_eta[mass < m_eta/2] * v_dl(mass[mass < m_eta / 2] ** 2 / m_eta ** 2) * charge[mass < m_eta / 2] ** 2

    sensitivity35m = pi + eta

    return sensitivity10m, sensitivity35m

In [ ]:
mass   = np.logspace(-3, 0, 500)
charge = np.logspace(-5, 1, 500)
masses, charges = np.meshgrid(mass, charge)

lanl10m_4layer, lanl35m_4layer = generateLANL(masses, charges, 4)
lanl10m_3layer, lanl35m_3layer = generateLANL(masses, charges, 3)
lanl10m_2layer, lanl35m_2layer = generateLANL(masses, charges, 2)
lanl10m_1layer, lanl35m_1layer = generateLANL(masses, charges, 1)
lanl10m_2lyaer_yield, lanl35m_2layer_yield = generateLANL(masses, charges, 2, 1.25e6)
lanl10m_1layer_yield, lanl35m_1layer_yield = generateLANL(masses, charges, 1, 1.25e6)

print("Generation completed") 

In [ ]:
figure(figsize=(22, 16), dpi=500)
plt.tick_params(axis='both', which='both', labelsize=25)

plt.contour(mass, charge, lanl10m_4layer, levels=[8], colors = 'greenyellow')
plt.plot(10, 5, color='greenyellow', label='4 layers, 80 bars/layer, bkg = 10')

plt.contour(mass, charge, lanl10m_3layer, levels=[46], colors = 'tomato')
plt.plot(10, 5, color='tomato', label='3 layers, 80 bars/layer, bkg = 10')

plt.contour(mass, charge, lanl10m_2layer, levels=[61], colors = 'skyblue')
plt.plot(10, 5, color='skyblue', label='2 layers, 80 bars/layer, bkg = 900')

plt.contour(mass, charge, lanl10m_2layer, levels=[20], colors = 'cyan')
plt.plot(10, 5, color='cyan', label='2 layers, 80 bars/layer, bkg = 90')

plt.contour(mass, charge, lanl10m_2lyaer_yield, levels=[61], colors='orange')
plt.plot(10, 5, color='orange', label='2 layers, 80 bars/layer, bkg = 900, High photon yield')

plt.contour(mass, charge, lanl10m_2lyaer_yield, levels=[20], colors='darkorange')
plt.plot(10, 5, color='darkorange', label='2 layers, 80 bars/layer, bkg = 90, High photon yield')

plt.contour(mass, charge, lanl10m_1layer_yield, levels=[428], colors='blueviolet')
plt.plot(10, 5, color='blueviolet', label='1 layer, 1 bar, bkg = 47000, High photon yield')

colliders_plot = plt.fill_between(colliders[0], colliders[1], 2, label = 'Colliders', alpha=0.5, color = 'mediumseagreen') #'green')
slac_plot = plt.fill_between(slac[0], slac[1], 2, label = 'SLAC', alpha=0.5, color = 'gold') #'orange')
bebc_plot = plt.fill_between(bebc[0], bebc[1], 2, label = 'BEBC', alpha=0.5, color = 'gray')
lsnd_plot = plt.fill_between(lsnd[0]/1000, lsnd[1], 2, label='LSND', alpha=0.5, color = 'skyblue')
charmii_plot = plt.fill_between(charmii[0], charmii[1], 2, label = 'Charm II', alpha=0.5, color = 'lightgray')
argonuet_plot = plt.fill_between(argoneut[0], argoneut[1], 2, label = 'ArgoNeuT', alpha=0.5, color = 'cornflowerblue') #color = 'mediumorchid')
mq_demo_plot = plt.fill_between(mq_demo[0], mq_demo[1], 2, label = 'MilliQan demonstrator', alpha=0.5, color = 'lightcoral')

plt.xscale('log')
plt.yscale('log')
plt.xlabel('$m_{\chi}$ [$\mathrm{GeV}/\mathrm{c}^2$]', fontsize = 35)
plt.xticks(fontsize = 35)
plt.ylabel('$\epsilon=Q/e$', fontsize = 35)
plt.yticks(fontsize = 35)
plt.xlim(0.0053, 10)
plt.ylim(0.000005, 1)

plt.legend(loc='lower right', fontsize=19.5)
#plt.savefig('sensitivity_lanl.png')
#plt.savefig('sensitivity_lanl.pdf')
plt.show()

In [ ]:
slac = pd.read_csv('slac.csv',header=None)
colliders = pd.read_csv('colliders.csv',header=None)
bebc = pd.read_csv('bebc.csv',header=None)
charmii = pd.read_csv('charmii.csv',header=None)
mq_demo = pd.read_csv('mq_demonstrator_sort.csv',header=None)       
argoneut = pd.read_csv('argoneut_sort.csv',header=None)
lsnd = pd.read_csv('LSND.csv', header=None)
lsnd.sort_values(by=lsnd.columns[0])

In [ ]:
np.shape(slac)
print(slac)

In [ ]:
figure(figsize=(22, 16), dpi=500)
plt.tick_params(axis='both', which='both', labelsize=25)

plt.contour(mass, charge, lanl35m_2layer, levels=[26], colors = 'cyan')
plt.plot(10, 5, color='cyan', label='ER2, 2 layers, 80 bars/layer, bkg = 380')

plt.contour(mass, charge, lanl35m_2layer, levels=[7943], colors = 'blue')
plt.plot(10, 5, color='blue', label='ER2, 2 layers, 80 bars/layer, bkg = 2E7'
        )
plt.contour(mass, charge, lanl10m_2layer, levels=[26], colors = 'red')
plt.plot(10, 5, color='red', label='ER1, 2 layers, 80 bars/layer, bkg = 380')

plt.contour(mass, charge, lanl10m_2layer, levels=[20177], colors = 'tomato')
plt.plot(10, 5, color='tomato', label='ER1, 2 layers, 80 bars/layer, bkg = 2E7')



colliders_plot = plt.fill_between(colliders[0], colliders[1], 2, label = 'Colliders', alpha=0.5, color = 'mediumseagreen') #'green')
slac_plot = plt.fill_between(slac[0], slac[1], 2, label = 'SLAC', alpha=0.5, color = 'gold') #'orange')
bebc_plot = plt.fill_between(bebc[0], bebc[1], 2, label = 'BEBC', alpha=0.5, color = 'gray')
lsnd_plot = plt.fill_between(lsnd[0]/1000, lsnd[1], 2, label='LSND', alpha=0.5, color = 'skyblue')
charmii_plot = plt.fill_between(charmii[0], charmii[1], 2, label = 'Charm II', alpha=0.5, color = 'lightgray')
argonuet_plot = plt.fill_between(argoneut[0], argoneut[1], 2, label = 'ArgoNeuT', alpha=0.5, color = 'cornflowerblue') #color = 'mediumorchid')
mq_demo_plot = plt.fill_between(mq_demo[0], mq_demo[1], 2, label = 'MilliQan demonstrator', alpha=0.5, color = 'lightcoral')

plt.xscale('log')
plt.yscale('log')
plt.xlabel('$m_{\chi}$ [$\mathrm{GeV}/\mathrm{c}^2$]', fontsize = 35)
plt.xticks(fontsize = 35)
plt.ylabel('$\epsilon=Q/e$', fontsize = 35)
plt.yticks(fontsize = 35)
plt.xlim(0.0053, 10)
plt.ylim(0.000005, 1)

plt.legend(loc='lower right', fontsize=19.5)
#plt.savefig('sensitivity_lanl.png')
#plt.savefig('sensitivity_lanl.pdf')
plt.show()

In [ ]:
figure(figsize=(22, 16), dpi=500)
plt.tick_params(axis='both', which='both', labelsize=25)

plt.contour(mass, charge, lanl35m_2layer, levels=[26], colors = 'cyan')
plt.plot(10, 5, color='cyan', label='ER2, 2 layers, 80 bars/layer, bkg = 380')

plt.contour(mass, charge, lanl35m_2layer, levels=[7943], colors = 'blue')
plt.plot(10, 5, color='blue', label='ER2, 2 layers, 80 bars/layer, bkg = 2E7')

plt.contour(mass, charge, lanl10m_2layer, levels=[26], colors = 'red')
plt.plot(10, 5, color='red', label='ER1, 2 layers, 80 bars/layer, bkg = 380')

plt.contour(mass, charge, lanl10m_2layer, levels=[20177], colors = 'tomato')
plt.plot(10, 5, color='tomato', label='ER1, 2 layers, 80 bars/layer, bkg = 2E7')



colliders_plot = plt.fill_between(colliders[0], colliders[1], 2, label = 'Colliders', alpha=0.5, color = 'mediumseagreen') #'green')
slac_plot = plt.fill_between(slac[0], slac[1], 2, label = 'SLAC', alpha=0.5, color = 'gold') #'orange')
bebc_plot = plt.fill_between(bebc[0], bebc[1], 2, label = 'BEBC', alpha=0.5, color = 'gray')
lsnd_plot = plt.fill_between(lsnd[0]/1000, lsnd[1], 2, label='LSND', alpha=0.5, color = 'skyblue')
charmii_plot = plt.fill_between(charmii[0], charmii[1], 2, label = 'Charm II', alpha=0.5, color = 'lightgray')
argonuet_plot = plt.fill_between(argoneut[0], argoneut[1], 2, label = 'ArgoNeuT', alpha=0.5, color = 'cornflowerblue') #color = 'mediumorchid')
mq_demo_plot = plt.fill_between(mq_demo[0], mq_demo[1], 2, label = 'MilliQan demonstrator', alpha=0.5, color = 'lightcoral')

plt.xscale('log')
plt.yscale('log')
plt.xlabel('$m_{\chi}$ [$\mathrm{GeV}/\mathrm{c}^2$]', fontsize = 35)
plt.xticks(fontsize = 35)
plt.ylabel('$Q_{\chi}/e$', fontsize = 35)
plt.yticks(fontsize = 35)
plt.xlim(0.0053, 10)
plt.ylim(0.000005, 1)

plt.legend(loc='lower right', fontsize=19.5)
#plt.savefig('sensitivity_lanl.png')
#plt.savefig('sensitivity_lanl.pdf')
plt.show()